In [1]:
from gensim.models.doc2vec import Doc2Vec

c:\program files\python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [54]:
# Format data for input into Doc2Vec
import nltk
import csv
from collections import namedtuple
from numpy.random import choice

ScoreDocument = namedtuple('ScoreDocument', 'words tags split score')
elements = ['train', 'test', 'validate'] 
weights = [0.7, 0.2, 0.1]

titles = []
with open('all_submissions.csv','r', encoding='utf-8') as infile:
    reader = csv.DictReader(infile)
    counter = 0
    for row in reader:
        title = row['title'].lower()
        if '[removed]' in title or '[deleted]' in title: continue
        selftext = row['selftext'].lower()
        if '[removed]' in selftext or '[deleted]' in selftext: continue
        words = nltk.word_tokenize(title + " " + selftext)
        score = int(row['ups']) - int(row['downs'])
        titles.append(ScoreDocument(words, tags=[counter], split=choice(elements, p=weights), score=score))
        counter += 1

In [55]:
train_docs = [doc for doc in titles if doc.split == 'train']
test_docs = [doc for doc in titles if doc.split == 'test']
val_docs = [doc for doc in titles if doc.split == 'validate']

print(len(train_docs))
print(len(test_docs))
print(len(val_docs))
# Access document vectors with test_model.docvecs[doc.tags[0]]

404513
115928
57287


In [56]:
import multiprocessing
cores = multiprocessing.cpu_count()
model = Doc2Vec(titles, size=100, workers=cores, window=8, min_count=5)
model.save('queries.doc2vec')

c:\program files\python36\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [69]:
import numpy as np
import statsmodels.api as sm
from random import sample

threshold = 1 #Can set this parameter to make more/fewer passes to the test. 0 is 89%, 1 is 0.28

def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    # print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set, infer=False, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[(1 if (doc.score > threshold) else 0, test_model.docvecs[doc.tags[0]]) for doc in train_set])
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    test_regressors = [test_model.docvecs[doc.tags[0]] for doc in test_docs]
    test_regressors = sm.add_constant(test_regressors)
    
    # Predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    rounded_predictions = np.rint(test_predictions)
    correct_set = rounded_predictions == [1 if (doc.score > threshold) else 0 for doc in test_data]
    false_pos = 0
    false_neg = 0
    for i in range(len(correct_set)):
        if rounded_predictions[i] == 1 and correct_set[i] == 0: false_pos += 1
        if rounded_predictions[i] == 0 and correct_set[i] == 0: false_neg += 1
    corrects = sum(correct_set)
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), false_pos, false_neg)

err, err_count, test_count, false_pos, false_neg = error_rate_for_model(model, train_docs, test_docs)
print("Error rate: " + str(err))
print("False positives: " + str(false_pos))
print("False negatives: " + str(false_neg))

Error rate: 0.26482816920847424
False positives: 5062
False negatives: 25639


In [70]:
print(sum([1 if (doc.score > 1) else 0 for doc in test_docs]))

32751


In [74]:
import random

doc_id = np.random.randint(model.docvecs.count)  # pick random doc, re-run cell for more examples
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(u'TARGET (%d): «%s»\n' % (doc_id, ' '.join(titles[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(titles[sims[index][0]].words)))

TARGET (105824): «when i put on my friends glasses , is what i see equivalent to what they see without their glasses ? i think everyone has wondered this at some point havent they ? is what i see while wearing my friends glasses equal in distortion as what they see without their glasses on . or if they are nearsighted , when i wear their glasses does my vision become as nearsighted as they are naturally ?»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d100,n5,w8,mc5,s0.001,t4):

MOST (304794, 0.8584342002868652): «if there were a group of healthy infant children on an island with a vocally mute caregiver to raise them , what would their vocalizations and communications be like ?»

MEDIAN (204492, 0.5854758620262146): «how do hormonal birth control pills change the behavior of women who use them ? do the altered amounts of estrogen and progesterone that occur while women use the pill produce a difference in behavior ? do birth control pills subdue certain behavioral patterns ? are the